In [ ]:
import numpy as np
import h5py
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs


import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-78,central_longitude=-20)#260-360)
axex = [275,332,-80,-76.5]

In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3000,6500),y=slice(4000,7000))
    mask = xr.where(ds.mask==1,2,ds.mask)
    #mask[:] = xr.where(mask==1,2,mask)
    ds = add_lonlat(ds)
    
    ax.set_facecolor(".7")
    
    cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.algae_r')(np.linspace(0,.4,255)))
    cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    #im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.surface-ds.thickness,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=0,vmax=2000)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    ax.set_extent(axex,crs=ccrs.PlateCarree())
    #gl = ax.gridlines(linewidth=.2,color='.9')
    #gl.xlocator = mpl.ticker.FixedLocator(np.arange(-180,179,1))
    #gl.ylocator = mpl.ticker.FixedLocator(np.arange(-89,89,.2))

In [ ]:
#Read observations Adusumilli

fname = '../../../data/adusumilli/bb0448974g_3_1.h5'
hf = h5py.File(fname, 'r')
xa = np.array(hf['/x'])[:,0]
ya = np.array(hf['/y'])[:,0]
melta = np.array(hf['/w_b'])


data_vars = {'melt':(['y','x'],melta)}
coords = {'x': (['x'], xa),'y': (['y'], ya)}

dsa = xr.Dataset(data_vars=data_vars,coords=coords)
dsa = dsa.isel(x=slice(2000,5000),y=slice(4500,7000))
dsa = add_lonlat(dsa)

In [ ]:
vmax = 10
vmin = -3
linthresh=.1
linscale=.1

vmax = 100
vmin = -10
linthresh = .3
linscale = .25

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#fracpos = (np.log10(vmax/linthresh)+.5*linscale)/(np.log10(vmax)+np.log10(-(vmin/linthresh))+1*linscale)

#print(fracpos)
nneg = np.int_((1-fracpos)*256)+1

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,1,nneg))
#colors1 = plt.cm.PuBu(np.linspace(.2,.8, nneg))
colors2 = plt.cm.inferno(np.linspace(0, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)

In [ ]:
plt.style.use('style_paper')
mpl.rc('figure.subplot',left=.01,right=.89,top=.99,bottom=0.,wspace=.05,hspace=.01)

ds = xr.open_dataset(f'../../results/FRIS_2.0_linear_S134.3_T1-2.3_720.nc')

fig = plt.figure(figsize=(7,3))

ax = fig.add_subplot(121,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(dsa.lon,dsa.lat,dsa.melt,shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())
#ax.gridlines(draw_labels=True)
ax.set_title('a) Observations',loc='left')

ax.text(.05,.37,'Evans\nIce\nStream',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.08,.16,'Carlson\nInlet',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.1,.09,'Rutford\nIce Stream',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.42,-.01,'Institute\nIce\nStream',rotation=-45,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.58,.03,'Möller\nIce\nStream',rotation=60,transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.75,.0,'Foundation\nIce Stream',rotation=-10,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.81,.12,'Support\nForce\nGlacier',rotation=-80,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.89,.43,'Recovery\nGlacier',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.94,.63,'S.Gl.',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.95,.72,'Bailey\nIce Stream',rotation=90,transform=ax.transAxes,fontsize=4,color='w')

ax.text(.7,.5,'Berkner\nIsland',ha='center',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.34,.27,'K.I.R.',rotation=-80,transform=ax.transAxes,fontsize=4,color='w')
#ax.text(.43,.26,'D.I.R.',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.53,.29,'H.I.R.',rotation=-60,transform=ax.transAxes,fontsize=4,color='w')

ax.text(.25,.8,'Ronne Ice Shelf',transform=ax.transAxes,color='w')
ax.text(.65,.8,'Filchner\nIce Shelf',transform=ax.transAxes,color='w')


ax = fig.add_subplot(122,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())
#ax.gridlines(draw_labels=True)
ax.set_title('b) LADDIE 2.0 km',loc='left')

axins = inset_axes(ax,width="5%",height="100%",loc='lower left',bbox_to_anchor=(1.05,0,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cbar = plt.colorbar(im, cax=axins,extend='both')
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
cbar.set_label('Freezing / Melt rate [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_FRIS_2.0.png',dpi=450)

In [ ]:
plt.style.use('style_paper')
mpl.rc('figure.subplot',left=.01,right=.89,top=.99,bottom=0.,wspace=.05,hspace=.01)

ds = xr.open_dataset(f'../../results/FRIS_1.0_linear_S134.3_T1-2.3_400.nc')

fig = plt.figure(figsize=(7,3))

ax = fig.add_subplot(121,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(dsa.lon,dsa.lat,dsa.melt,shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())
#ax.gridlines(draw_labels=True)
ax.set_title('a) Observations',loc='left')

ax.text(.05,.37,'Evans\nIce\nStream',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.08,.16,'Carlson\nInlet',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.1,.09,'Rutford\nIce Stream',transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.42,-.01,'Institute\nIce\nStream',rotation=-45,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.58,.03,'Möller\nIce\nStream',rotation=60,transform=ax.transAxes,fontsize=4,color='w',ha='center')
ax.text(.75,.0,'Foundation\nIce Stream',rotation=-10,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.81,.12,'Support\nForce\nGlacier',rotation=-80,transform=ax.transAxes,fontsize=4,color='w')
ax.text(.89,.43,'Recovery\nGlacier',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.94,.63,'S.Gl.',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.95,.72,'Bailey\nIce Stream',rotation=90,transform=ax.transAxes,fontsize=4,color='w')

ax.text(.7,.5,'Berkner\nIsland',ha='center',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.34,.27,'K.I.R.',rotation=-80,transform=ax.transAxes,fontsize=4,color='w')
#ax.text(.43,.26,'D.I.R.',transform=ax.transAxes,fontsize=4,color='w')
ax.text(.53,.29,'H.I.R.',rotation=-60,transform=ax.transAxes,fontsize=4,color='w')

ax.text(.25,.8,'Ronne Ice Shelf',transform=ax.transAxes,color='w')
ax.text(.65,.8,'Filchner\nIce Shelf',transform=ax.transAxes,color='w')

ax = fig.add_subplot(122,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())
#ax.gridlines(draw_labels=True)
ax.set_title('b) LADDIE 1.0 km',loc='left')

axins = inset_axes(ax,width="5%",height="100%",loc='lower left',bbox_to_anchor=(1.05,0,1,1),bbox_transform=ax.transAxes,borderpad=0)
#ticks = [-3,-1,-.3,-.1,0,.1,.3,1,3,10]
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cbar = plt.colorbar(im, cax=axins,extend='both')
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
cbar.set_label('Freezing / Melt rate [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_FRIS_1.0.png',dpi=450)